<a href="https://colab.research.google.com/github/JinHyun-P/HelloGit/blob/master/AI_01_%E1%84%87%E1%85%A1%E1%86%A8%E1%84%8C%E1%85%B5%E1%86%AB%E1%84%92%E1%85%A7%E1%86%AB_N224a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 2 / NOTE 4*

# 📝 Assignment
---

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


target = 'vacc_h1n1_f'
# target = 'vacc_seas_f'
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [2]:
# 훈련데이터를 훈련/검증 데이터로 나누기

train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)


train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [4]:
# 특성공학

def engineer(df):

  # behavioral 컬럼 통합하기
  
  behaviorals = [col for col in df.columns if 'behavioral' in col]
  df['behaviorals'] = df[behaviorals].sum(axis=1)

  # 높은 카디널리티 가지는 특성 제거

  selected_cols = df.select_dtypes(include=['number', 'object'])
  labels = selected_cols.nunique() # 특성별 카디널리티 리스트
  selected_features = labels[labels <= 30].index.tolist() # 카디널리티가 30보다 작은 특성만 선택
  
  # 상관없는 데이터 삭제

  dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
  df.drop(columns=dels, inplace=True)

  return df

train = engineer(train)
val = engineer(val)
test = engineer(test)

In [5]:
# 데이터에서 타겟과 특성 분리

features = train.drop(columns=[target]).columns

In [6]:
# 훈련/검증/테스트 데이터를 특성과 타겟으로 분리

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [13]:
# !pip install category_encoders

In [11]:
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 모델선택(Model Selection)

### 1) 캐글 대회를 이어서 진행합니다. RandomizedSearchCV 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.

주의해야 할 점은, 이번에는 [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)를 사용한다는 점입니다.

또한 분류문제에서 맞는 [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values) 같은 metric을 사용해야 합니다.

그리고 인코더 사용에도 주의 해야 하는데, 지금 다루는 분류 문제에서는 [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html)사용을 추천합니다.


In [12]:
from sklearn.model_selection import RandomizedSearchCV
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint, uniform

pipe = make_pipeline(
    TargetEncoder(), 
    SimpleImputer(), 
    RandomForestRegressor(random_state=2)
)

dists = {
    'targetencoder__smoothing': [2.,20.,50.,60.,100.,500.,1000.], # int로 넣으면 error(bug)
    'targetencoder__min_samples_leaf': randint(1, 10),     
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(100, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    #'randomforestregressor__max_features': uniform(0, 1) # max_features
}

clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='neg_mean_absolute_error',  
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train, y_train);

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 57.9min finished
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [14]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('MAE: ', -clf.best_score_)

최적 하이퍼파라미터:  {'randomforestregressor__max_depth': 10, 'randomforestregressor__n_estimators': 242, 'simpleimputer__strategy': 'mean', 'targetencoder__min_samples_leaf': 1, 'targetencoder__smoothing': 50.0}
MAE:  0.2498114708357595


In [19]:
# 각 하이퍼파라미터 모델들 순위별로 나열

pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').T

,25,15,27,17,40,23,14,20,11,47,34,24,45,12,0,29,1,16,39,32,46,31,21,7,3,37,33,22,8,26,4,35,5,48,2,42,6,19,18,43,30,38,9,44,41,10,13,36,49,28
mean_fit_time,31.4534,55.7421,55.6367,62.4537,58.4264,63.0372,62.5256,20.4431,38.4495,18.7128,42.0744,13.4821,19.919,27.0231,39.2065,37.2036,32.4397,48.52,69.5897,79.0459,77.7448,17.0617,39.4789,8.50832,13.2706,7.75873,16.0278,26.7208,67.6163,33.5822,19.361,23.3586,35.4004,30.27,46.426,63.8026,68.7943,66.5836,77.0123,45.0936,63.2293,63.2064,67.2405,61.63,45.1617,86.9902,49.6466,50.753,61.8648,40.9381
std_fit_time,0.424569,0.187859,0.45665,1.75265,0.45804,0.246395,0.253583,0.0469282,0.487115,0.173874,0.815974,0.141508,0.0158306,0.233884,0.961659,1.10828,0.416098,0.258811,0.433115,2.82206,0.691849,0.267688,0.418564,0.0799884,0.093593,0.0990425,0.248607,0.221727,0.788864,0.320052,0.267377,0.484465,0.418694,0.541973,0.550611,0.470794,0.691229,0.58755,0.298761,0.487491,0.949443,0.347516,0.68329,0.170139,0.435053,0.373996,0.227444,0.232415,7.48483,0.330603
mean_score_time,0.772504,1.36362,1.15144,1.45067,0.813866,1.46024,1.54251,0.496655,0.951452,0.337259,0.860469,0.224165,0.31853,0.383719,0.676702,0.798674,0.586864,1.16838,1.27882,1.79798,1.37096,0.461817,0.989322,0.174136,0.217456,0.153252,0.262597,0.419331,1.75979,0.845174,0.476575,0.621866,0.856194,0.60497,0.881477,1.18796,1.87558,1.84536,2.09854,0.858572,1.7644,1.40705,1.70461,1.18867,0.870536,2.35697,1.39304,1.38121,1.30743,1.09684
std_score_time,0.0108714,0.0173312,0.0722374,0.102506,0.0261774,0.131571,0.0494456,0.0442974,0.0321336,0.0230631,0.179785,0.0126627,0.0342575,0.00895355,0.00882858,0.0333836,0.0337765,0.109987,0.105699,0.176043,0.0573177,0.00966811,0.0508017,0.0089648,0.00609356,0.00272752,0.00603878,0.00570137,0.0802765,0.10752,0.076392,0.0774572,0.136162,0.0191573,0.069087,0.0355607,0.0564046,0.0139631,0.00730764,0.0299363,0.0075913,0.0947252,0.082981,0.0603689,0.0224479,0.0294484,0.0367937,0.0780717,0.0660615,0.061287
param_randomforestregressor__max_depth,10,10,10,10,10,10,10,10,15,15,15,5,5,5,20,20,20,20,None,None,None,10,10,5,5,5,5,5,15,15,20,20,20,20,20,20,20,20,20,20,20,None,None,None,None,None,None,None,None,None
param_randomforestregressor__n_estimators,242,421,436,486,499,480,477,152,226,116,262,186,306,385,228,208,190,258,396,434,443,141,318,129,223,114,275,412,424,213,114,135,214,186,289,396,394,378,439,277,370,375,376,365,272,481,273,283,412,228
param_simpleimputer__strategy,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median,median
param_targetencoder__min_samples_leaf,1,7,8,6,7,6,9,6,1,4,3,4,5,9,6,3,3,6,1,6,4,5,1,1,3,8,7,4,1,6,2,6,7,4,4,6,5,4,5,1,4,3,8,4,8,7,7,2,9,9
param_targetencoder__smoothing,50,20,60,20,2,50,60,1000,60,20,100,50,500,2,100,2,2,2,500,2,1000,20,50,100,50,100,50,1000,100,1000,60,50,500,500,2,1000,50,500,1000,60,500,500,60,1000,2,100,2,20,50,20
params,"{'randomforestregressor__max_depth': 10, 'rand...","{'randomforestregressor__max_depth': 10, 'rand...","{'randomforestregressor__max_depth': 10, 'rand...","{'randomforestregressor__max_depth': 10, 'rand...","{'randomforestregressor__max_depth': 10, 'rand...","{'randomforestregressor__max_depth': 10, 'rand...","{'randomforestregressor__max_depth': 10, 'rand...","{'randomforestregressor__max_depth': 10, 'rand...","{'randomforestregressor__max_depth': 15, 'rand...","{'randomforestregressor__max_depth': 15, 'rand...","{'randomforestregressor__max_depth': 15, 'rand...","{'randomforestregressor__max_depth': 5, 'rando...","{'randomforestregressor__max_depth': 5, 'rando...","{'randomforestregressor__max_depth': 5, 'rando...","{'randomforestregressor__max_depth': 20, 'rand...","{'randomforestregressor__max_depth': 20, 'rand...","{'randomforestregressor__max_depth': 20, 'rand...","{'randomforestregressor__max_dep

In [20]:
# 만들어진 모델에거 가장 성능이 좋은 모델을 불러옵니다.

pipe = clf.best_estimator_


### 2) (도전과제🔥) [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.
